# Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:

id | user_id | venue_id | latitude | longitude | created_at

---------+---------+----------+-------------------+-------------------+---------------------

984301 | 2041916 | 5222 | | | 2012-04-21 17:39:01

984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47

984315 | 1764391 | 5222 | | | 2012-04-21 17:37:18

984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43

...

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

...

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

<b>Примечание:</b> на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

38.8951118,-77.0363658

33.800745,-84.41052

45.5234515,-122.6762071

...

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

<b>Примечание:</b> при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift

In [2]:
data = pd.read_csv('checkins.csv', delimiter='|')
data.drop(0, inplace=True)
data.drop([data.columns[0], data.columns[1], data.columns[2], data.columns[5]], axis=1, inplace=True)
data.dropna(axis=0, inplace=True)
data.replace('                   ', np.nan, inplace=True)
data.dropna(inplace=True)

/Volumes/Storage/Applications/Programming/anaconda3/envs/py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head(10)

,latitude,longitude
2,38.8951118,-77.0363658
4,33.800745,-84.41052
8,45.5234515,-122.6762071
10,40.764462,-111.904565
11,33.4483771,-112.0740373
12,32.2217429,-110.926479
13,40.65,-73.95
14,33.4483771,-112.0740373
16,33.414768,-111.9093095
19,42.3584308,-71.0597732


In [4]:
ms = MeanShift(bandwidth=0.1)

In [5]:
ms.fit(data[0:100000])
labels = ms.labels_
cluster_centers = ms.cluster_centers_

In [6]:
classters_list = np.zeros((1, len(cluster_centers)))[0]
for i in range(0, len(cluster_centers)):
    classters_list[i] = len(np.where(labels == i)[0])

In [7]:
classters_more_15_inx = np.where(classters_list > 15)
classters_more_15_centers = cluster_centers[classters_more_15_inx]

In [8]:
file = open('centers.txt', 'w')
for center in classters_more_15_centers:
    file.write(str(center[0]) + ',' + str(center[1]) + '\n')
file.close

<function close>

In [9]:
offices_coordinates = (np.array([[33.751277, -118.188740],
                                 [25.867736, -80.324116],
                                 [51.503016, -0.075479],
                                 [52.378894, 4.885084],
                                 [39.366487, 117.036146],
                                 [-33.868457, 151.205134]]))

In [10]:
centers_min_distacence = np.zeros((1, len(classters_more_15_centers)))[0]
for i, center in enumerate(classters_more_15_centers):
    distances_to_offices = np.array([(center[0] - office[0]) ** 2 + 
                                     (center[1] - office[1]) ** 2 
                                     for office in offices_coordinates])
    centers_min_distacence[i] = np.min(distances_to_offices)

In [11]:
zipped_array = zip(classters_more_15_centers, centers_min_distacence)

In [12]:
sorted(zipped_array, key = lambda t: t[1])

[(array([-33.86063043, 151.20477593]), 6.138343547438518e-05),
 (array([52.37296399,  4.89231722]), 8.748452367514415e-05),
 (array([ 25.84567226, -80.3188906 ]), 0.0005141132761548423),
 (array([51.50299126, -0.12553729]), 0.0025058328805651507),
 (array([  33.80987796, -118.14892381]), 0.005019401190076088),
 (array([ 25.78581242, -80.21793804]), 0.01798523282924887),
 (array([ 25.70534972, -80.28342874]), 0.028024756866587088),
 (array([ 26.01009825, -80.19999059]), 0.03567412849327191),
 (array([  33.88832534, -118.04892817]), 0.03832959557867868),
 (array([  33.87298601, -118.36209115]), 0.04486370350982694),
 (array([  33.97257482, -118.16837067]), 0.04938763550948061),
 (array([ 26.13884379, -80.33434684]), 0.07360410210889788),
 (array([  33.98393587, -118.00740497]), 0.08701254365803676),
 (array([ 26.12086266, -80.15890668]), 0.09136722465435418),
 (array([  33.81730643, -117.89124917]), 0.09286067950641896),
 (array([  34.06039755, -118.24870903]), 0.09915180141653804),
 (ar

In [13]:
sorted_centers_min_distacence = np.sort(centers_min_distacence)

In [14]:
sorted_centers_min_distacence[-21:-1]

array([1617.36900191, 1619.85036427, 1622.1873192 , 1728.45195659,
       1738.28158287, 1760.0455563 , 1930.7995501 , 2178.39742913,
       2181.33475459, 2183.24726149, 2187.03726688, 2191.28028485,
       2230.69515505, 2268.23346691, 3278.03361991, 3547.17746483,
       3576.80469181, 3610.84471829, 3756.1255647 , 3776.15052151])